In [2]:
import os
import subprocess
from google.colab import drive

# === CẤU HÌNH RIÊNG (CHỈ THAY 1 LẦN) ===
USERNAME = "FongNgoo"   # ← THAY ĐỔI
REPO_NAME = "Basic_Dynamic_Prices_base_on_Demand_Model"        # ← THAY ĐỔI
BRANCH = "main"                     # hoặc "master"
# ======================================

REPO_DIR = f"/content/{REPO_NAME}"
REPO_URL = f"git@github.com:{USERNAME}/{REPO_NAME}.git"

!git config --global user.email "phongngo4060@gmail.com"
!git config --global user.name "FongNgoo"

In [3]:
def _setup_ssh():
    """Tải SSH key từ Drive và cấu hình"""
    drive.mount('/content/drive', force_remount=False)
    SSH_DIR = '/content/drive/MyDrive/.colab_ssh'
    SSH_KEY = f"{SSH_DIR}/id_colab"
    if not os.path.exists(SSH_KEY):
        raise FileNotFoundError("SSH key chưa tồn tại! Chạy file setup trước.")

    os.makedirs('/root/.ssh', exist_ok=True)
    subprocess.run(['cp', f'{SSH_KEY}', f'{SSH_KEY}.pub', '/root/.ssh/'], check=False)
    subprocess.run(['chmod', '600', '/root/.ssh/id_colab'], check=True)
    subprocess.run(['chmod', '644', '/root/.ssh/id_colab.pub'], check=True)

    config = f"""Host github.com
    HostName github.com
    User git
    IdentityFile /root/.ssh/id_colab
    StrictHostKeyChecking no
"""
    with open('/root/.ssh/config', 'w') as f:
        f.write(config)
    subprocess.run(['chmod', '600', '/root/.ssh/config'], check=True)

In [4]:
def _ensure_repo():
    """Đảm bảo repo tồn tại, clone nếu cần"""
    if not os.path.exists(REPO_DIR):
        print(f"Repo không tồn tại → Đang clone: {REPO_URL}")
        result = subprocess.run(['git', 'clone', REPO_URL, REPO_DIR])
        if result.returncode != 0:
            raise RuntimeError("Clone thất bại! Kiểm tra SSH key và repo URL.")
    # Luôn cd vào repo
    os.chdir(REPO_DIR)
    print(f"Đã vào thư mục: {REPO_DIR}")

In [5]:
import os
import subprocess
from datetime import datetime
from google.colab import drive

BRANCH = "main"   # đổi thành "master" nếu repo bạn dùng master

def push_only(message=None, force=False):
    """
    CHỈ PUSH – KHÔNG PULL, KHÔNG REBASE, KHÔNG HỎI
    Dùng khi bạn chắc chắn muốn đẩy hết mọi thứ trong Colab lên GitHub ngay lập tức
    """
    repo_path = "/content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model"

    # 1. Fix Drive treo (bắt buộc)
    if not os.path.exists('/content/drive/MyDrive'):
        print("Drive chưa mount → mount lại...")
        drive.mount('/content/drive', force_remount=True)
    else:
        try:
            os.listdir('/content/drive/MyDrive')
        except:
            print("Drive bị treo → kill và remount...")
            !pkill -9 -f "drive" 2>/dev/null
            drive.mount('/content/drive', force_remount=True)

    # 2. Vào thư mục repo
    os.chdir(repo_path)
    print(f"Đã vào: {os.getcwd()}")

    # 3. Kiểm tra nhanh có gì thay đổi không
    status = subprocess.check_output(['git', 'status', '--porcelain']).decode().strip()
    if not status:
        print("Không có thay đổi nào → Không cần push.")
        return

    print(f"Phát hiện thay đổi:\n{status}")

    # 4. Commit
    msg = message or f"Quick push từ Colab - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    print(f"Đang commit: {msg}")
    !git add . --all
    !git commit -m "{msg}"

    # 5. Push (có lựa chọn force)
    if force:
        print("Đang FORCE PUSH (--force-with-lease)...")
        !git push origin {BRANCH} --force-with-lease
    else:
        print("Đang push bình thường...")
        result = subprocess.run(['git', 'push', 'origin', BRANCH])
        if result.returncode != 0:
            print("Push thường bị từ chối (có xung đột) → tự động chuyển sang force-with-lease")
            !git push origin {BRANCH} --force-with-lease

    print("PUSH XONG 100% LÊN GITHUB!")

In [ ]:
push_only()

Đã vào: /content/drive/MyDrive/Colab_Notebooks/Basic_Dynamic_Prices_base_on_Demand_Model
